In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-5e028c07-beab-aa4b-1d93-73c1f1b1f681)


In [ ]:
%cd drive/My\ Drive/Colab\ Notebooks/apex-codes/entity_sum

/content/drive/My Drive/Colab Notebooks/apex-codes/entity_sum


In [ ]:
!pip3 install -q transformers

     |████████████████████████████████| 3.4 MB 8.0 MB/s 
     |████████████████████████████████| 596 kB 73.8 MB/s 
     |████████████████████████████████| 67 kB 6.7 MB/s 
     |████████████████████████████████| 895 kB 73.4 MB/s 
     |████████████████████████████████| 3.3 MB 73.5 MB/s 


In [ ]:
!pip3 install -q biobert-embedding
!pip3 install -q gensim

     |████████████████████████████████| 748.9 MB 653 bytes/s 
     |████████████████████████████████| 123 kB 92.8 MB/s 
     |████████████████████████████████| 131 kB 90.2 MB/s 
     |████████████████████████████████| 79 kB 10.2 MB/s 
     |████████████████████████████████| 8.5 MB 64.9 MB/s 
     |████████████████████████████████| 138 kB 88.4 MB/s 
     |████████████████████████████████| 127 kB 102.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.2.0 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.2.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.2.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
import nltk
nltk.download('punkt')  
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import gensim
import os
import pandas as pd
import numpy as np
import pickle as pk
import torch
from scipy.spatial.distance import cosine
import scipy.spatial as sp
from gensim.test.utils import common_texts
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import sent_tokenize

In [ ]:
import numpy as np
import pandas as pd
import pickle as pk
import os
from sklearn.cluster import KMeans
from sklearn import metrics
from collections import defaultdict
import json
from biobert_embedding.embedding import BiobertEmbedding

In [ ]:
biobert = BiobertEmbedding()

## Semantic Equivalence between summaries and abstracts

In [ ]:
# This one uses BioBERT
def _compute_cosine_score(gen_summary, human_summary):
  gen_summary_embeddings = biobert.word_vector(gen_summary)
  human_summary_embeddings = biobert.word_vector(human_summary)

  lst_gen_summary_embed = [x.numpy() for x in gen_summary_embeddings]  # conversion into numpy array for each tensor
  lst_human_summary_embed = [x.numpy() for x in human_summary_embeddings]    # conversion into numpy array

  cosine = sp.distance.cdist(lst_gen_summary_embed, lst_human_summary_embed, 'cosine')

  avg_cosine = np.sum(cosine.flatten()) / (len(lst_gen_summary_embed) * len(lst_human_summary_embed))

  return avg_cosine

  
  '''
  total_combinations = len(gen_summary_embeddings) * len(human_summary_embeddings)
  cosine_sum = 0.0

  for gen_summary_embedding in gen_summary_embeddings:
    for human_summary_embedding in human_summary_embeddings:
      cosine_sum += cosine(gen_summary_embedding, human_summary_embedding)

  avg_cosine_sum = cosine_sum / float(total_combinations)

  return avg_cosine_sum
  '''


In [ ]:
def main():
  lst_modelName = ["BART"]
  lst_model_type = ["vanilla", "w_named_entities"]
  

  dict_cosine_scores = defaultdict(list)

  for modelName in lst_modelName:
    for model_type in lst_model_type:
      SUMMARY_PATH = f"pubmed-FINAL-SUMMARIES/{modelName}"
      input_filename_path = f"{SUMMARY_PATH}/pubmed-summaries-{model_type}.jsonl"

      cosine_sum = 0.0   # initialize all cumulative scores to zero
      total_no_summaries = 0
      with open(input_filename_path) as fp:
        for iter, line in enumerate(fp):
          if iter % 2000 == 0:
            print("Iteration: ", iter)
          dict_data = json.loads(line)

          generated_summary = dict_data["abstractive_summary"]    # generated summary
          ground_truth_summary = dict_data["article_abstract"]   # human-like summary

          try:
            gen_summary_embed = biobert.word_vector(generated_summary)
            human_summary_embed = biobert.word_vector(ground_truth_summary)

            # call to the semantic equivalence computing method
          
            cosine_score = _compute_cosine_score(generated_summary, ground_truth_summary)

            cosine_sum += cosine_score
            
            total_no_summaries += 1

          except:
            continue

      fp.close()

      avg_cosine = cosine_sum/float(total_no_summaries) * 100

      dict_cosine_scores["training-config"].append(f"{modelName}-{model_type}")
      dict_cosine_scores["cosine-score"].append(avg_cosine)

      print(dict_cosine_scores)

  df_cosine_scores = pd.DataFrame(dict_cosine_scores)

  print(df_cosine_scores)

  df_cosine_scores.to_excel("cosine_scores.xlsx")  

In [ ]:
if __name__ == "__main__":
  main()

Iteration:  0
Iteration:  2000
Iteration:  4000
defaultdict(<class 'list'>, {'training-config': ['BART-vanilla'], 'cosine-score': [51.799856080051434]})
Iteration:  0
Iteration:  2000
Iteration:  4000
defaultdict(<class 'list'>, {'training-config': ['BART-vanilla', 'BART-w_named_entities'], 'cosine-score': [51.799856080051434, 51.783337061770574]})
         training-config  cosine-score
0           BART-vanilla     51.799856
1  BART-w_named_entities     51.783337
